# 8. LangGraph로 만드는 AI 에이전트
## ⑶ 따라하기: Q&A 애플리케이션

In [1]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-test"

In [3]:
ROLES = {
    "1": {
        "name": "일반지식 전문가",
        "description": "다양한 분야의 일반적인 질문에 대한 답변을 제공",
        "details": "다양한 분야의 일반적인 질문에 대해 정확하고 이해하기 쉬운 답변을 제공해야 합니다."
    },
    "2": {
        "name": "생성형 AI 제품 전문가",
        "description": "생성형 AI 및 관련 제품, 기술에 대한 전문적인 질문에 대한 답변을 제공",
        "details": "생성형 AI 및 관련 제품, 기술에 대한 전문적인 질문에 대한 최신 정보와 깊은 통찰력을 제공하십시오."
    },
    "3": {
        "name": "카운슬러",
        "description": "개인적 고민과 심리적 문제에 대한 지원 제공",
        "details": "개인적 고민이나 심리적 문제에 대해 공감하고 지지적인 답변을 제공하고, 가능하다면 적절한 조언을 제공해야 합니다."
    }
}

In [5]:
import operator
from typing import Annotated

from pydantic import BaseModel, Field


class State(BaseModel):
    query: str = Field(..., description="사용자들의 질문")
    current_role: str = Field(
        default="", description="선정된 답변 규칙"
    )
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="답변 이력"
    )
    current_judge: bool = Field(
        default=False, description="품질 검사 결과"
    )
    judgement_reason: str = Field(
        default="", description="품질검사 판정 이유"
    )

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
# 나중에 max_tokens의 값을 변경할 수 있도록 변경가능한 필드를 선언함
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [7]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
"""질문을 분ㅅ헉하고 가장 적합한 답변 담당 역할을 선택하세요.

선택:
{role_options}

답은 선택지 번호(1,2,3)으로만 답해 주십시오.

질문: {query}
""".strip()
    )
    # 선택지 번호만 반환하기를 원하므로 max_tokens의 값을 1로 변경함
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [8]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
"""당신은 {role}로 답변해 주세요. 다음 질문에 대해 귀하의 역할에 따라 적절한 답변을 제공하십시오.

역할상세:
{role_details}

질문: {query}

답변:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [9]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="判定結果")
    reason: str = Field(default="", description="判定理由")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""아래 답변의 품질을 확인하고, 문제가 있으면 'False', 문제가 없으면 'True'로 답해 주십시오.
또한, 그 판단의 이유를 설명해 주십시오.

사용자들의 질문: {query}
답변: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [10]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [11]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [12]:
# selection노드에서 처리 시작
workflow.set_entry_point("selection")

In [13]:
# selection노드에서 answering노드까지
workflow.add_edge("selection", "answering")
# answering노드에서 check노드까지
workflow.add_edge("answering", "check")

In [14]:
from langgraph.graph import END

# check 노드에서 다음 노드로의 전환을 위한 조건부 에지를 정의
# state.current_judge의 값이 True이면 END 노드로, False이면 selection 노드로 이동
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [15]:
compiled = workflow.compile()

In [16]:
initial_state = State(query="생성형 AI에 대해 알려주세요")
result = compiled.invoke(initial_state)

In [ ]:
result

In [17]:
print(result["messages"][-1])

생성형 AI 제품 전문가로서 답변드리겠습니다.

생성형 AI는 인공지능의 한 분야로, 주어진 데이터를 기반으로 새로운 콘텐츠를 생성하는 기술을 말합니다. 이 기술은 자연어 처리, 이미지 생성, 음악 작곡 등 다양한 분야에서 활용되고 있습니다. 대표적인 예로는 OpenAI의 GPT-3와 같은 대규모 언어 모델이 있으며, 이는 텍스트 기반의 다양한 작업을 수행할 수 있습니다.

생성형 AI의 핵심은 대량의 데이터를 학습하여 패턴을 인식하고, 이를 바탕으로 새로운 데이터를 생성하는 능력입니다. 이러한 AI 모델은 주로 딥러닝 기술을 활용하며, 특히 인공신경망을 사용하여 복잡한 데이터 구조를 이해하고 생성합니다.

최근에는 생성형 AI가 더욱 발전하여, 예술 작품을 창작하거나, 프로그래밍 코드를 작성하는 등 창의적인 작업에서도 두각을 나타내고 있습니다. 또한, 이러한 기술은 개인화된 콘텐츠 생성, 자동화된 고객 서비스, 게임 개발 등 다양한 산업에서 혁신을 이끌고 있습니다.

생성형 AI의 발전은 많은 기회를 제공하지만, 동시에 윤리적 문제와 데이터 사용에 대한 책임도 중요하게 고려되어야 합니다. AI가 생성한 콘텐츠의 신뢰성, 저작권 문제, 그리고 편향성 문제 등이 주요 이슈로 대두되고 있습니다. 따라서 생성형 AI를 개발하고 활용할 때는 이러한 문제들을 충분히 인식하고 해결하기 위한 노력이 필요합니다.


## ⑷ 체크포인트 기능: 상태 영속화 및 재개기능

In [18]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22 langgraph-checkpoint==1.0.11

  Attempting uninstall: langgraph-checkpoint
    Found existing installation: langgraph-checkpoint 1.0.12
    Uninstalling langgraph-checkpoint-1.0.12:
      Successfully uninstalled langgraph-checkpoint-1.0.12


In [19]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-test"

In [20]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 그래프상태 정의
class State(BaseModel):
    query: str
    messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

# 메시지를 추가하는 노드 함수
def add_message(state: State) -> dict[str, Any]:
    additional_messages = []
    if not state.messages:
        additional_messages.append(
            SystemMessage(content="당신은 최소한의 응답을 하는 대화형 에이전트입니다.")
        )
    additional_messages.append(HumanMessage(content=state.query))
    return {"messages": additional_messages}

# LLM의 응답을 추가하는 노드 함수
def llm_response(state: State) -> dict[str, Any]:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    ai_message = llm.invoke(state.messages)
    return {"messages": [ai_message]}

In [21]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("체크포인트 데이터:")
    pprint(checkpoint_tuple.checkpoint)
    print("\n메타데이터:")
    pprint(checkpoint_tuple.metadata)

In [22]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# 그래프 설정
graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

# 체크포인터 설정
checkpointer = MemorySaver()

# 그래프 컴파일
compiled_graph = graph.compile(checkpointer=checkpointer)

In [23]:
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.")
first_response = compiled_graph.invoke(user_query, config)
first_response

{'query': '제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화형 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다! 떡볶이를 좋아하시는군요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 48, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-63e0c4c1-891b-4006-b2a6-1c1c6708d676-0', usage_metadata={'input_tokens': 48, 'output_tokens': 14, 'total_tokens': 62})]}

In [24]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1efa4d7c-c526-6bfc-8002-3b7170cfef4f'}}, checkpoint={'v': 1, 'ts': '2024-11-17T11:33:42.465206+00:00', 'id': '1efa4d7c-c526-6bfc-8002-3b7170cfef4f', 'channel_values': {'query': '제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.', 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화형 에이전트입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='알겠습니다! 떡볶이를 좋아하시는군요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 48, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finis

In [25]:
print_checkpoint_dump(checkpointer, config)

체크포인트 데이터:
{'channel_values': {'llm_response': 'llm_response',
                    'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화형 에이전트입니다.', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='알겠습니다! 떡볶이를 좋아하시는군요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 48, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-63e0c4c1-891b-4006-b2a6-1c1c6708d676-0', usage_metadata={'input_tokens': 48, 'output_tokens': 14, 'total_tokens': 62})],
 

In [26]:
user_query = State(query="내가 좋아하는 음식이 무엇인지 기억나세요?")
second_response = compiled_graph.invoke(user_query, config)
second_response

{'query': '내가 좋아하는 음식이 무엇인지 기억나세요?',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화형 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='알겠습니다! 떡볶이를 좋아하시는군요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 48, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-63e0c4c1-891b-4006-b2a6-1c1c6708d676-0', usage_metadata={'input_tokens': 48, 'output_tokens': 14, 'total_tokens': 62}),
  HumanMessage(content='내가 좋아하는 음식이 무엇인지 기억나세요?', additional_kwargs={}, response_metadata={}),
  AIMessage(co

In [27]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1efa4d87-d14e-6bf1-8006-8d5e672e255d'}}, checkpoint={'v': 1, 'ts': '2024-11-17T11:38:39.018881+00:00', 'id': '1efa4d87-d14e-6bf1-8006-8d5e672e255d', 'channel_values': {'query': '내가 좋아하는 음식이 무엇인지 기억나세요?', 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화형 에이전트입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='알겠습니다! 떡볶이를 좋아하시는군요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 48, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reaso

In [28]:
print_checkpoint_dump(checkpointer, config)

체크포인트 데이터:
{'channel_values': {'llm_response': 'llm_response',
                    'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화형 에이전트입니다.', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='제가 가장 좋아하는 것, 떡볶이입니다. 기억해 두세요.', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='알겠습니다! 떡볶이를 좋아하시는군요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 48, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-63e0c4c1-891b-4006-b2a6-1c1c6708d676-0', usage_metadata={'input_tokens': 48, 'output_tokens': 14, 'total_tokens': 62}),
  

In [29]:
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="내가 좋아하는 음식은?")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response

{'query': '내가 좋아하는 음식은?',
 'messages': [SystemMessage(content='당신은 최소한의 응답을 하는 대화형 에이전트입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='내가 좋아하는 음식은?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='모르겠어요. 어떤 음식을 좋아하세요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 37, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-89f32620-8984-4f23-9d9b-5f600165eb2d-0', usage_metadata={'input_tokens': 37, 'output_tokens': 11, 'total_tokens': 48})]}